In [4]:
from rich import print
from sqlalchemy import create_engine, text
from sqlalchemy import MetaData, Table, Column
from sqlalchemy import Integer, String, ForeignKey
from sqlalchemy import select, insert, delete, update

In [5]:
engine = create_engine(
    'sqlite:///:memory:',
    echo=True,
)

In [6]:
with engine.connect() as conn:
    resule = conn.execute(text('select sqlite_version()'))
    print(resule.one())

2025-09-04 23:32:14,279 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:32:14,280 INFO sqlalchemy.engine.Engine select sqlite_version()
2025-09-04 23:32:14,280 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()


('3.47.1',)

2025-09-04 23:32:14,312 INFO sqlalchemy.engine.Engine ROLLBACK


## Create Table

In [7]:
meta = MetaData()

In [8]:
users_table = Table(
    'users',
    meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('login', String(20), unique=True),
    Column('name', String(50)),
)

cart_table = Table(
    'cart',
    meta,
    Column('id', Integer, primary_key=True),
    Column('user_id', Integer, ForeignKey('users.id')),
    Column('product_id', Integer),
    Column('quantity', Integer),
)

meta.create_all(engine)

2025-09-04 23:32:14,322 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:32:14,323 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-09-04 23:32:14,323 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-04 23:32:14,324 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-09-04 23:32:14,324 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-04 23:32:14,325 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cart")
2025-09-04 23:32:14,325 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-04 23:32:14,326 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cart")
2025-09-04 23:32:14,326 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-04 23:32:14,327 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	login VARCHAR(20), 
	name VARCHAR(50), 
	PRIMARY KEY (id), 
	UNIQUE (login)
)


2025-09-04 23:32:14,327 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2025-09-04 23:32:14,328 INFO sqlalchemy.engine.Engine 
CREATE TA

In [9]:
print(repr(users_table))

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), 
Column('login', String(length=20), table=<users>), Column('name', String(length=50), table=<users>), schema=None)

In [10]:
print(meta.tables)

FacadeDict({'users': Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, 
nullable=False), Column('login', String(length=20), table=<users>), Column('name', String(length=50), 
table=<users>), schema=None), 'cart': Table('cart', MetaData(), Column('id', Integer(), table=<cart>, 
primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<cart>), 
Column('product_id', Integer(), table=<cart>), Column('quantity', Integer(), table=<cart>), schema=None)})

## Insert values

In [11]:
add_users = insert(users_table).values(
    [
        {users_table.c.login: 'john', users_table.c.name: 'John Doe'},
        {users_table.c.login: 'jane', users_table.c.name: 'Jane Doe'},
    ]
)

with engine.connect() as conn:
    conn.execute(add_users)
    conn.commit()

2025-09-04 23:32:14,347 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:32:14,348 INFO sqlalchemy.engine.Engine INSERT INTO users (login, name) VALUES (?, ?), (?, ?)
2025-09-04 23:32:14,348 INFO sqlalchemy.engine.Engine [no key 0.00104s] ('john', 'John Doe', 'jane', 'Jane Doe')
2025-09-04 23:32:14,349 INFO sqlalchemy.engine.Engine COMMIT


## Get value(s)

In [12]:
from rich.table import Table

with engine.connect() as conn:
    res = conn.execute(
        select(
            users_table.c.login,
            users_table.c.name,
        ).order_by(users_table.c.login),
    )

    users = res.all()
    # print(res.one())
    # print(res.scalar())

    table = Table(title='Users')
    table.add_column('login')
    table.add_column('name')

    for login, name in users:
        table.add_row(login, name)

    print(table)


2025-09-04 23:32:14,355 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:32:14,355 INFO sqlalchemy.engine.Engine SELECT users.login, users.name 
FROM users ORDER BY users.login
2025-09-04 23:32:14,356 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ()


       Users        
┏━━━━━━━┳━━━━━━━━━━┓
┃ login ┃ name     ┃
┡━━━━━━━╇━━━━━━━━━━┩
│ jane  │ Jane Doe │
│ john  │ John Doe │
└───────┴──────────┘

2025-09-04 23:32:14,358 INFO sqlalchemy.engine.Engine ROLLBACK


## Delete values

In [ ]:
delete_stmt = delete(users_table).where(users_table.c.login == 'jane')
select_stmt = select(users_table.c.login, users_table.c.name)

with engine.connect() as conn:
    conn.execute(delete_stmt)
    conn.commit()

    res = conn.execute(select_stmt)

print(res.all())

2025-09-04 23:32:59,684 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:32:59,685 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.login = ?
2025-09-04 23:32:59,685 INFO sqlalchemy.engine.Engine [cached since 44.48s ago] ('jane',)
2025-09-04 23:32:59,686 INFO sqlalchemy.engine.Engine COMMIT


0

2025-09-04 23:32:59,687 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:32:59,688 INFO sqlalchemy.engine.Engine SELECT users.login, users.name 
FROM users
2025-09-04 23:32:59,688 INFO sqlalchemy.engine.Engine [cached since 44.48s ago] ()
2025-09-04 23:32:59,688 INFO sqlalchemy.engine.Engine ROLLBACK


[('john', 'John Doe')]

## Update values

In [17]:
update_stmt = update(users_table).where(users_table.c.login == 'john').values(
    name='John J. Doe',
)

with engine.connect() as conn:
    conn.execute(update_stmt)
    conn.commit()

    res = conn.execute(select_stmt)

print(res.all())

2025-09-04 23:33:58,704 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:33:58,705 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.login = ?
2025-09-04 23:33:58,706 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ('John J. Doe', 'john')
2025-09-04 23:33:58,707 INFO sqlalchemy.engine.Engine COMMIT
2025-09-04 23:33:58,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-04 23:33:58,708 INFO sqlalchemy.engine.Engine SELECT users.login, users.name 
FROM users
2025-09-04 23:33:58,708 INFO sqlalchemy.engine.Engine [cached since 104.2s ago] ()
2025-09-04 23:33:58,709 INFO sqlalchemy.engine.Engine ROLLBACK


[('john', 'John J. Doe')]